In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [49]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *
from utilities.misc import *
from utilities.simplifier import Simplifier
from utilities.discrimination import *
from utilities.idinserter import IdInserter
from utilities.evaluator import Evaluator
from utilities.gate_killer import GateKiller
from ast import literal_eval
import tensorflow_quantum as tfq

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


kill 1qbit gate, try 0/4. Increased by: 0.0%
kill 1qbit gate, try 0/4. Increased by: 0.0%


In [64]:
translator = CirqTranslater(3, untouchable_blocks = [1], discard_qubits=[2])
simplifier = Simplifier(translator)
inserter = IdInserter(translator.n_qubits, untouchable_blocks=translator.untouchable_blocks, untouchable_qubits = [2])


etas = [0.2,1.]
minimizer = Minimizer(translator, mode="discrimination", params=etas)
killer = GateKiller(translator, mode="discrimination", params=etas)

args_evaluator = {"n_qubits":translator.n_qubits, "problem":"acd","params":etas}
evaluator = Evaluator(args=args_evaluator, lower_bound_cost=minimizer.lower_bound_cost, nrun=0, stopping_criteria=1e-3)


### prepare initial circuit ####
channel_db = amplitude_damping_db(translator, qubits_ind=[0,inserter.untouchable_qubits[0]], eta=1, block_id = translator.untouchable_blocks[0])
circuit_db = concatenate_dbs([x_layer(translator,qubits_ind=[0,1], block_id=0), channel_db, x_layer(translator,qubits_ind=[0,1], block_id=2)])
circuit, circuit_db = translator.give_circuit(circuit_db)


# optimize continuous parameters ##

minimized_db, [cost, resolver, history_training] = minimizer.variational(circuit_db)
evaluator.add_step(minimized_db, cost, relevant=True, operation="variational", history = history_training.history["cost"])

### reduce circuit (same cost) ###
simplified_db, ns =  simplifier.reduce_circuit(minimized_db)
newcost = minimizer.give_cost(simplified_db)
reduced_db, reduced_cost, ops = kill_and_simplify(simplified_db, newcost, killer, simplifier)
evaluator.add_step(simplified_db, reduced_cost, relevant=True, operation="reduction", history = ops)



circuit_db = reduced_db.copy()


for vans_it in range(evaluator.vans_its):
    print("vans_it: {}\n current cost: {}\ntarget cost: {} \nrelative error: {}\n\n\n".format(vans_it, cost, minimizer.lower_bound_cost, (cost-minimizer.lower_bound_cost)/np.abs(minimizer.lower_bound_cost)))
    mutated_db, number_mutations = inserter.mutate(circuit_db)
    mutated_cost = minimizer.give_cost(mutated_db)
    evaluator.add_step(mutated_db, mutated_cost, relevant=False, operation="mutation", history = number_mutations)

    simplified_db, ns =  simplifier.reduce_circuit(mutated_db)
    simplified_cost = minimizer.give_cost(simplified_db)
    evaluator.add_step(simplified_db, simplified_cost, relevant=False, operation="simplification", history = ns)

    minimized_db, [cost, resolver, history_training] = minimizer.variational(simplified_db)
    evaluator.add_step(minimized_db, cost, relevant=False, operation="variational", history = history_training.history["cost"])

    accept_cost, stop = evaluator.accept_cost(cost)
    if accept_cost == True:

        reduced_db, reduced_cost, ops = kill_and_simplify(simplified_db, cost, killer, simplifier)
        evaluator.add_step(reduced_db, reduced_cost, relevant=False, operation="reduction", history = ops)

        minimized_db, [cost, resolver, history_training] = minimizer.variational(reduced_db)
        evaluator.add_step(minimized_db, cost, relevant=True, operation="variational", history = history_training.history["cost"])

        circuit_db = minimized_db.copy()
        
    if stop == True:
        print("ending VAns")
        print("\n final cost: {}\ntarget cost: {} \n\n\n\n".format(cost, minimizer.lower_bound_cost))
        break

vans_it: 0
 current cost: 0.16569793224334717
target cost: 0.1656980511955941 
relative error: -7.194368549789942e-07



ending VAns

 final cost: 0.16569799184799194
target cost: 0.1656980511955941 




